Created on Tue Mar 19 15:37 2019

Figure showing thickness evolution of snow and ice for ARC3O Paper part 1 (Figure 2)

@author: Clara Burgard

In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
sns.set_context('paper')

READ IN THE DATA

In [7]:
inputpath_FYI = '/work/mh0033/m300411/SatSim/data_repo_part1/MEMLS_input_output/exp049_75N00W_20191009/INPUT/netcdf_files/'
inputpath_MYI = '/work/mh0033/m300411/SatSim/data_repo_part1/MEMLS_input_output/exp050_NorthPole_20191011/INPUT/netcdf_files/'
inputpath_samsim = '/work/mh0033/m300411/SatSim/data_repo_part1/SAMSIM_input_output/SAMSIM_OUTPUT/'
outputpath_fig = 'path/to/plots'

In [8]:
memls_input_FYI = xr.open_dataset(inputpath_FYI+'inputMEMLS_depthidx_75N00W.nc')
timelength_FYI = memls_input_FYI['time']
ts_month_FYI = timelength_FYI['time.month']

thick_sn_file_FYI = np.loadtxt(inputpath_samsim+'75N00W/dat_snow.dat') #thick_snow,T_snow,psi_l_snow,psi_s_snow
thick_sn_FYI = thick_sn_file_FYI[:,0]
thick_sn_FYI[thick_sn_FYI==0] = np.nan

memls_input_FYI['thick_snow'] = xr.DataArray(thick_sn_FYI, coords=[timelength_FYI], dims=['time'])

memls_input_MYI = xr.open_dataset(inputpath_MYI+'inputMEMLS_depthidx_NorthPole.nc')
timelength_MYI = memls_input_MYI['time']
ts_month_MYI = timelength_MYI['time.month']
                              
thick_sn_file_MYI = np.loadtxt(inputpath_samsim+'NorthPole/dat_snow.dat') #thick_snow,T_snow,psi_l_snow,psi_s_snow
thick_sn_MYI = thick_sn_file_MYI[:,0]
thick_sn_MYI[thick_sn_MYI==0] = np.nan

memls_input_MYI['thick_snow'] = xr.DataArray(thick_sn_MYI, coords=[timelength_MYI], dims=['time'])

COMPUTE THE TOTAL THICKNESS FROM THE LAYERS

In [ ]:
#First-year ice
tot_thick0 = np.zeros(len(timelength_FYI)) #timelength is 3285
for tt,timet in enumerate(timelength_FYI):
    #print(tt)
    a = memls_input_FYI['thickness'].sel(sens_exp='complex',time=timet).dropna(dim='depth',how='any')
    if a.size:
        tot_thick0[tt] = a.depth.max()  
    else:
        tot_thick0[tt] = np.nan  
tot_thick_FYI = xr.DataArray(tot_thick0,dims=memls_input_FYI['thickness'].sel(sens_exp='complex',depth=0.0).dims)      



In [ ]:
#Multiyear ice 
tot_thick1 = np.zeros(len(timelength_MYI)) #timelength is 3285
for tt,timet in enumerate(timelength_MYI):
    #print(tt)
    a = memls_input_MYI['thickness'].sel(sens_exp='complex',time=timet).dropna(dim='depth',how='any')
    if a.size:
        tot_thick1[tt] = a.depth.max()  
    else:
        tot_thick1[tt] = np.nan  
tot_thick_MYI = xr.DataArray(tot_thick1,dims=memls_input_MYI['thickness'].sel(sens_exp='complex',depth=0.0).dims)      
          

Mask out the thickness peaks around snow melting

In [ ]:
#First-year ice
tot_thick_FYI.plot() 
#FYI: 654,687;1368,1407;2116,2146;2805,2879

In [ ]:
#Multiyear ice
tot_thick_MYI.plot() 
#MYI: 685,716;1420,1440;2152,2177;2874,2903

In [ ]:
timerange_ok_FYI = np.array([i for j in (range(0,655),range(687,1369),range(1407,2117),range(2146,2806),range(2879,3285)) for i in j])
timerange_ok_MYI = np.array([i for j in (range(0,685),range(716,1421),range(1440,2153),range(2177,2874),range(2903,3285)) for i in j])

timerange_notok_FYI = np.array([i for j in (range(655,687),range(1369,1407),range(2117,2146),range(2806,2879)) for i in j])
timerange_notok_MYI = np.array([i for j in (range(685,716),range(1421,1440),range(2153,2177),range(2874,2903)) for i in j])

In [ ]:
tot_thick_FYI_masked = tot_thick_FYI.copy()
tot_thick_FYI_masked[timerange_notok_FYI] = np.nan

tot_thick_MYI_masked = tot_thick_MYI.copy()
tot_thick_MYI_masked[timerange_notok_MYI] = np.nan

PLOT THE TIME EVOLUTION OF THE ICE AND SNOW HEIGHT

In [ ]:
fig, ax = plt.subplots(1, 2, sharex=True, sharey=True,figsize=(8,8/2))
ax[0].plot(timelength_FYI,tot_thick_FYI_masked*-1,'k-')
ax[0].plot(timelength_FYI,memls_input_FYI['thick_snow'],'k--')
ax[0].axhline(y=0,c='k',linestyle=':')
fig.autofmt_xdate()
ax[0].set_ylabel('Sea ice depth and snow thickness [m]')

ax[1].plot(timelength_MYI,tot_thick_MYI_masked*-1,'k-')
ax[1].plot(timelength_MYI,memls_input_MYI['thick_snow'],'k--')
ax[1].axhline(y=0,c='k',linestyle=':')
fig.autofmt_xdate()

sns.despine()
plt.tight_layout()
#fig.savefig(outputpath_fig+'Figure2.pdf',bbox_inches='tight')